# 유형1 : 데이터 전처리 3문제 * 10 [30점]
- pandas를 활용한 데이터 탐색 작업에 대한 능력 확인
- 주어진 데이터를 활용해 조건에 맞는 데이터를 추출하고 해당 데이터에 대한 통계량 구하거나 개수 세는 작업 요구
- 1개의 정수로 답 print()  <- 언제든지 바뀔 수 있음

In [ ]:
# 소수점 아래 2째자리까지 표시되도록 설정하기
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

## 1. 데이터 불러오기

In [ ]:
import pandas as pd

# csv 파일 불러오기 
df = pd.read_csv('파일경로.csv')
## 불러오면서 인덱스 컬럼 설정
df_csv1 = pd.read_csv('파일경로.csv', index_col='인덱스로 삼을 칼럼명')
## 불러오면서 구분자 지정
df_csv2 = pd.read_csv('파일경로.csv', sep='\t')
## 불러올 때 언어 설정
df_csv3 = pd.read_csv('파일경로.csv', encoding='euc-kr')  # 한글지원 : euc-kr, cp949
## 불러올 때 결측치 필터 사용 끄기
df_csv2 = pd.read_csv('파일경로.csv', na_filter=False)


# excel 파일 불러오기 (엑셀은 엔진 설정해주기!!)
df_excel = pd.read_excel('파일경로.xlsx', engine='openpyxl')

# csv 파일 저장하기
df.to_csv('경로.csv', index=False)
# excel 파일 저장하기
df.to_excel('경로.xlsx', index=False)

## 2. 데이터 구조 확인 

In [ ]:
# 데이타가 많은 경우 모두 출력 안되고 ... 으로 생략해서 출력됩니다.
# 생략되지 않는 행, 열의 개수를 설정하여 생략되지 않고 출력되도록 합니다.
pd.set_option('display.max_rows', 800)    #출력할 max row를 지정
pd.set_option('display.max_columns', 100)  #출력할 max columns를 지정

In [ ]:
# 처음부터 n개 행의 데이터 확인
df.head()
# 끝부터 n개 행의 데이터 확인
df.tail()

# 데이터프레임 row개수, colum개수, Not null,dtype 등 정보 확인
df.info(memory_usage='deep')
# 데이터프레임 통계 정보 확인
df.describe()

# 데이터프레임의 행, 열의 수를 (행, 열)형태의 튜블로 반환
df.shape
# 데이터프레임의 (행 x 열)의 전체 데이터 수를 반환
df.size
# 데이터프레임 데이터 타입 확인
df.dtypes

# 시리즈
sri = df['특정 컬럼']
# 시리즈 데이터 타입 확인
sri.dtypes

#데이터프레임의 인덱스 확인 (보통 0으로 시작해서 몇으로 끝나고 스텝이 몇인지 등을 보여줌)
df.index
# 데이터프레임의 컬럼 확인
df.columns          # 타입 index
df.columns.values   # 타입 array
# 데이터프레임의 구성요소 2차원으로 보기
df.values           # 타입 array

# 시리즈의 구성요소
sri.values          # 타입 array 해당 시리즈 값 배열로 쭉 나옴

In [ ]:
# 시리즈 중 컬럼에 존재하는 값 종류만 확인(결측치 포함, ndrarray로 반환)
sri.unique()       
# 시리즈 중 컬럼에 존재하는 값 종류의 개수만 확인
sri.nunique()
# 시리즈 중 컬럼에 대해 값 별 개수 확인(결측치 미포함, Series 반환)
sri.value_counts() 

# 해당 칼럼의 결측치 개수 까지 같이 보고 싶은 경우
sri.value_counts(dropna=False)
# 해당 값들의 비율을 보고싶은 경우(결측치 비율까지 같이 나옴)
sri.value_counts(normalize=True, dropna=False)

## 3. 데이터 타입 변경
데이터 타입 변경 전 데이터 조작이 필요할 수 있음 (예) 불필요 문자/콤마/공백 제거, 단위 변환 등

In [ ]:
# 시리즈 데이터 타입 변경 ('int', 'int32', 'int64', 'float', 'str', 'category' 등)
## 넘파이 타입으로 하려면 넘파이 임포트 하여야함 (np.int16, np.float32, np.datetime64 등)
sri.astype('타입')

# 원하는 자료형으로 바꾸지 못하는 장애요소 제거하고 변환
## replace 사용 시 regex=True 옵션을 사용하면 일부 내용만 변경대상으로 지정할 수 있음
## 여기서 regex는 '정규표현식'을 의미
sri.replace('변경전', '변경후', regex=True)
# 변경할 내용이 두가지 이상일 때 list나 dic을 사용하여 변환
list1 = ['변경전1', '변경전2']
list2 = ['변경후1', '변경후2']
sri.replace(list1, list2, regex=True)
dic = {'변경전1':'변경후1', '변경전2':'변경후2'}
sri.replace(dic, regex=True).astype('int64')

## ,콤마는 메타 문자이므로 제거하고자 하는 경우 \, 처럼 역슬레시 사용
## 메타문자 종류 , . + ? * ^ $ 등
sri.replace(['\메타문자', '변견전2'], '', regex=True)  # 변경후 모습이 같다면 하나만 적어도 됨

Series는 Accessor라는 것을 가지고 있다. Accessor를 사용하기 위해서라도 데이터 타입 변경 필요
- dt : Datetime, Timedelta, Period
- str : String
- cat : Categorical 

In [ ]:
#Accessor 사용 .str accessor 사용하면 각 행에 문자열 처럼 접근 
sri.str[1:-1].astype('category')

일반적인 np.datetime64 나 category 타입 변경은 astype으로 변경 가능, 특수한 경우는 아래 방법 사용

In [ ]:
# datetime
import numpy as np
sri.astype(np.datetime64)  # 일/월/년 순으로 보기도 함 이런 경우 to_datatime을 사용
# 내가 원하는 방식으로 날짜를 읽어오고 싶을 때
## %Y: 4글자 년도, %y : 2글자 년도, %m : 2글자 월, %d : 2글자 일
pd.to_datetime(sri, format='%y-%m-%d')

# category
sri.astype('category')  # 이렇게 변환한 자료에 sort_values 적용하면 가나다 순 정렬됨 
# 카테고리 순서를 직접 지정하고 싶을 때
temp = pd.Categorical(sri, categories=["범주1", "범주2"], ordered=True)  # 이건 지금 시리즈는 아님
sri = temp  # 이건 시리즈
sri.sort_values() # 원하는 순서대로 정렬 할 수 있음

## 4. 데이터 정렬

In [ ]:
# sort_index() : 인덱스 정렬
# [3-24] '측정일시'를 index로 설정하고,
# index 기준으로 오름차순 정렬해서 df1으로 이름 붙입니다.
# 그래프에서 y축으로 사용하려고 합니다.
df1 = df.set_index('측정일시').sort_index()

# sort_values() : 밸류 정렬, 디폴트(내림차순)
df.sort_values('정렬기준칼럼', ascending=False)
# 오름차순인 경우
df.sort_values('정렬기준칼럼', ascending=True)
# 정렬 기준이 여러개인 경우
df.sort_values(['1차기준', '2차기준'], ascending=[False, False])  
# 정렬 기준이 여러개, 내림차순 오름차순도 여러개인 경우
df.sort_values(['1차기준', '2차기준'], ascending=[True, False])

DataFrame의 index, columns 및 Series의 index 는 대입연산을 사용하여 변경 가능 다만, 개수가 동일해야 함  
  
value는 인덱스를 이용해서 변경 가능 

In [ ]:
# 컬럼 개수 확인 -> 컬럼 이름 리스트를 대입 연산자로 넣기(단, 개수 동일하게)
df.columns = ['컬럼1', '컬럼2', '컬럼3']

컬럼이름 변경하기
- DataFrame.rename(columns={'변경전이름':'변경후이름', ...})
- DataFrame.rename({'변경전이름':'변경후이름', ...}, axis=1)

In [ ]:
# [4-17] bread의 '상세영업상태코드'라는 컬럼명을 '상태코드'로 변경한 뒤,
# 첫 2개의 행을 확인합니다.
bread = bread.rename(columns={'상세영업상태코드':'상태코드'})
bread.head(2)

- Series.argmax() : 가장 값이 큰 것의 integer index 구하기
- Series.argmin() : 가장 값이 작은 것의 integer index 구하기
- Series[Series.argmax()] : 가장 큰 값 구하기
- Series[Series.argmin()] : 가장 작은 값 구하기

인덱싱 할 때 레이블을 쓰면 ['전':'후'] 에서 후 까지 포함  
레이블이 아닌 숫자를 쓰면 [1:9] 뒤에 9 미포함

## 5. 데이터 통계 확인

### 1) 통계 함수
- axis=0 : 기본 값으로 행을 이동하면서, 행과 행의 연산을 수행한다.(수직으로 연산)
- axis=1 : 컬럼을 이동하며 컬럼과 컬럼의 연산을 수행한다.(수평으로 연산)

In [ ]:
# 기초 통계량 확인
df.describe()

# 개수 세기
df.count()
# 합계
df.sum()
# 노적
df.cumsum()
# 평균
df.mean()
# 표준편차 
df.std()
# 분산
df.var()
# 중앙값
df.median()
# 최빈값
df.mode()  # 시리즈로 나옴
# 최빈값 1개
df.mode()[0]
# 최대값
df.max()   # 최대값, 최소값은 문자열에 대해서도 반응하는데 이는 ord(문자) 코드 숫자 기반
# 최소값
df.min()

# 분위수
df.quantile([0.25, 0.5, 0.75])    # 1사분위 0.25, 2사분위 0.5, 3사분위 0.75, 
# IQR
df.quantile(0.75) - sri.quantile(0.25)
# 상위20% 이상, 하위 20%도 이하 모두 이상이하로 계산한다.


In [ ]:
# 통계 함수 여러개 적용하고 싶을 때 1
df.apply(["min", "max", "mean"])    # apply 사용하면 어떤 함수든 적용 가능
# 통계 함수 여러개 적용하고 싶을 때 2
df.agg(["min", "max", "mean"])

### 2) 그룹별 통계
- df.groupby(그룹명).통계함수() : 적용가능한 모든 단위
- df.groupby(그룹명)[칼럼].통계함수 : Series 단위
- df.groupby(그룹명)[[[칼럼1, 칼럼2 ... ]].통계함수 : 특정 컬럼 단위  
- 그룹별로 통계치 구할 땐 agg(['var', 'std', 'mode'])로 여러개 가능

* 통계 함수는 numerical 값에 적용되기 때문에 object는 형변환이 필요함

In [ ]:
# [2-54] 대륙별 주류 소비량 중앙값을 계산해 봅니다.
df.groupby('대륙').median()

# [2-55] 대륙별 맥주 소비량 평균은?
df.groupby('대륙')['맥주'].mean()

# [2-56] 전세계 맥주 소비량 평균보다 많은 맥주를 소비하는 대륙은?
temp = df.groupby('대륙')[['맥주']].mean()
temp[temp['맥주']> df['맥주'].mean()]

# 그룹바이 기준이 두개 이상인 경우 멀티 인덱스로 작업
# [3-34] df_dust에서 '년', '월'별 '미세먼지(㎍/㎥)' 데이터의 평균을 구해
# DataFrame으로 만들어 meandf 라는 이름을 지정합니다.ㅣ
meandf = df_dust.groupby(['년', '월'])[['미세먼지(㎍/㎥)']].mean()
meandf.head()

# [3-35] meandf에서 2017년 6월까지의 데이터만 출력합니다.  
meandf.loc[:(2017,6),:] # 멀티인덱스 인경우 튜플로 표시한다.

In [ ]:
# [3-39] df_dust의 일자(년, 월, 일)별 '미세먼지(㎍/㎥)'의 평균을 구합니다.
# 인덱스가 유지되지 않음
df_dust.groupby(['측정일시'])['미세먼지(㎍/㎥)'].mean()  # <- 동일한 값을 갖음.  MultiIndex 아님
# 인덱스가 유지되면서 그룹별 함수 적용
df_dust.groupby(['측정일시'])['미세먼지(㎍/㎥)'].transform('mean')

### 3) 피벗 테이블 : 행, 열 모두에 그룹으 지정하여 통계값 구하기
- df.pivo_table(index=행방향칼럼, columns=열방향칼럼, values=집계대상칼럼, aggfunc=통계함수)
- index, columns는 범주형, values는 연속형 사용

* df.pivot_table(index=행방향그룹열이름, columns=열방향그룹열이름, values=집계대상열이름, aggfunc=통계함수)
* index, columns는 범주형, values는 연속형 사용
* values, aggfunc의 경우 단독의 경우 출력에 표시되지 않으나, 목록은 표시됨

In [ ]:
# [2-58] pivot_table을 사용하여 대륙별(index), '맥주'와 '와인'의 mean, median, max 값을 구합니다.
df.pivot_table(index='대륙', values=['맥주', '와인'], aggfunc=['mean', 'median', 'max'])

# [2-59] pivot_table을 사용하여 대륙별(columns), '맥주'와 '와인'의 mean, median값을 구합니다.
df.pivot_table(columns='대륙', values=['맥주', '와인'], aggfunc=['mean', 'median', 'max'])

# [2-60] groupby를 사용하여 대륙별, '맥주'와 '와인'의 mean, median, max 값을 구합니다.
df.groupby('대륙')['맥주', '와인'].agg(['mean', 'median', 'max'])
# 이건 [2-58] 과 비슷하지만 피벗테이블을 쓰느냐 그룹바이를 쓰느냐에 따라 데이터프레임 구조가 미묘하게 다르다.

# [3-48] df_dust의 월/년 별 미세먼지의 'mean', 'min', 'max' 구하기
# pivot_table 사용, values의 경우 목록으로 지정시와 단독 지정시가 다르게 표시됨
df_dust.pivot_table(index='월', columns='년', values='미세먼지(㎍/㎥)', aggfunc=['mean', 'min', 'max'])

# [3-49] df_dust에서 '측정소명'이 '강남구'인 데이터의
# 월별(index), 년별(columns), 미세먼지 농도 평균을 조회하여 temp로 저장합니다
temp = df_dust.loc[df['측정소명']=='강남구'].pivot_table(index='월', columns='년', values=['미세먼지(㎍/㎥)'], aggfunc=['mean'])

In [ ]:
# [3-50] 2016년 ~ 2020년도 미세먼지 농도가 가장 높은 월의 위치
temp = df_dust.pivot_table(index='월', columns='년', values='미세먼지(㎍/㎥)', aggfunc='mean')
for year in temp.columns:
    idx=temp[year].argmax()
    print(temp.index[idx])

# [3-52] 2016년 ~ 2019년 월별 미세먼지 평균을 구해 temp (DataFrame)로 저장합니다.
temp = df_dust.loc[df_dust['년']<=2019].groupby('월')[['미세먼지(㎍/㎥)']].mean() 

# [3-52] 2016년 ~ 2019년 월별 미세먼지 평균을 구해 temp (DataFrame)로 저장합니다.
temp = df_dust.loc[df_dust['년']<=2019].groupby('월')[['미세먼지(㎍/㎥)']].mean()

### 4) Index, Columns 상호변경


- Columns to Index : df.set_index(['인덱스로 이동시킬 컬럼 명'..])
- Index to Columns : df.reset_index()

In [ ]:
# [2-32] 국가별 주류 소비량 합계(맥주, 증류주, 와인의 합)를 구해 봅니다.
df.set_index('국가')[['맥주', '증류주', '와인']].sum(axis=1)

# [2-33] df를 ['대륙', '국가']를 index로 지정하고, 대륙별, 국가명으로  정렬하여 df로 저장합니다.
df = df.set_index(['대륙','국가']).sort_index()

# [2-34] df의 index를 모두 columns로 이동합니다.
df = df.reset_index()
df.head()

## 6. 데이터 정제하기

### 1) 결측치
- nan : 넘파이 배열에서 결측치 나타내는 경우
- NaN : 판다스에서 시리즈나 데이터프레임에서 결측치 나타내는 경우 

#### (1) 결측치 확인

In [ ]:
# 결측치 찾기
df.isna()
df.isnull()
# 결측치 합계 구하기
df.isna().sum()

# 결측치가 아닌 것 찾기
df.notna()
df.notnull()

# 불리언 인덱싱으로 결측치만 찾기
df[df['컬럼'].isna()]

- 범주형 데이터 : 다른 범주로 만들어 채우기
- 연속형 데이터 : 0으로 채우기, 평균값으로 채우기, 범주별 평균값으로 채우기

In [ ]:
# 데이터 프레임의 결측치 채우기 (모두 같은 값으로)
df = df.fillna('대체값')

# 특정한 하나의 컬럼 결측치 채우기
df['칼럼'] = df['칼럼'].fillna('대체값')
df.loc[df['칼럼'].isna(), '대륙'] = '대체값'

DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
- 결측치 제거에 사용되는 메서드
- how='any' : 결측치가 하나라도 포함된 행 삭제
- how='all' : 모든 데이터가 결측치인 행 삭제
- axis=1 : 컬럼에 대해 동작
- thresh=숫자 : 숫자 이상의 데이터를 가진 행은 삭제 안함
- subset=[컬럼이름1, ...] : subset으로 지정된 컬럼만 사용하여 삭제 대상 검색

#### (2) 결측치 제거

In [2]:
# 결측치 제거
df.dropna()

# [3-30] df_dust 에서 ['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)']에서
# 모든 데이터가 결측치인 행을 제거하여 결과를 temp1으로 저장합니다
temp1 = df_dust.dropna(how='all', axis=0, subset=['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)'])

# [3-32] df_dust 에서 ['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)']에서
# 2개 이상의 데이터를 가진 행은 제거하지 않은 결과를 temp3로 저장합니다.
# (= 3개의 정보 중 1개의 데이터만 가진 행을 제거함)
temp3 = df_dust.dropna(thresh=2, axis=0, subset=['오존농도(ppm)','미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)'])

#### (3) 결측치 대체
Series.mask(조건, 조건이 참일 때 사용할 값 또는 값 목록)
- 조건이 True인 것에 대해 다른 값을 변경합니다.
- sri.isna() : NA값에 대해 True, NA아닌 것은 False

Series.where(조건, 조건이 거짓일 때 사용할 값 또는 값 목록)
- 조건이 False인 것에 대해서 다른 값으로 변경합니다.
- sri.notna() : NA값에 대해 False, NA아닌 것은 True

In [ ]:
# temp의 'A' 열에 대해서 결측치인 경우 'B'의 값으로 대체합니다.
temp['A'].mask(temp['A'].isna(), temp['B'])
# temp의 'A' 열에 대해서 결측치인 경우 'C'의 값으로 대체합니다.
temp['A'].where(temp['A'].notna(), temp['C'])

# [3-43] meandf에 '결측치대체' 및 '차이2'라는 컬럼을 추가합니다.
# '결측치대체' 컬럼은 df_dust에서 '년', '월'별 '미세먼지(㎍/㎥)' 데이터의 평균을 사용합니다.
# '차이2' 컬럼은 '미세먼지(㎍/㎥)' - '결측치대체'를 사용합니다.
meandf['결측치대체'] = df_dust.groupby(['년','월'])['미세먼지(㎍/㎥)'].mean()

# [3-45] df_dust의 '오존농도(ppm)', '초미세먼지(㎍/㎥)' 컬럼에 대해서도
# '미세먼지(㎍/㎥)'와 같이 동일한 '년', '월', '일'의 평균 값으로 채우기 합니다.
fine_dust = df_dust.groupby('측정일시')['오존농도(ppm)'].transform('mean')
s = df_dust['오존농도(ppm)']
df_dust['오존농도(ppm)'] = s.mask(s.isna(), fine_dust)

fine_dust = df_dust.groupby('측정일시')['초미세먼지(㎍/㎥)'].transform('mean')
s = df_dust['초미세먼지(㎍/㎥)']
df_dust['초미세먼지(㎍/㎥)'] = s.mask(s.isna(), fine_dust)

### 2) 이상치

#### (1) 이상치 확인

In [ ]:
import matplotlib.pyplot as plt

# 시리즈의 이상치를 확인하는 박스플롯 그리기
sri.plot.box(figsize=(3,4))
plt.show()

# 데이터프레임의 연속형 데이터에 대해 모두 박스플롯 그리기
df.plot(kind='box', subplots=True)
plt.show()

# [5] ESD(Extream Studentized Diviate)를 이용한 방법
# 평균으로 부터 3 표준편차 떨어진 값을 이상치로 판단
# tip에 대한 이상치 구하기, 소수점 아래 2째자리까지 표기
s = tips['tip']
s_mean, s_std = s.mean(), s.std()
e_lower =  round(s_mean - 3 * s_std, 2) # round(숫자, 소수이하표시자리)
e_upper =  round(s_mean + 3 * s_std, 2)
print(f'Lower: {e_lower}, Upper: {e_upper}')
# 소수점 아래 2째자리 까지 표기는 출력시에만 그렇지 않으면 이상치 대체할 때 박스플롯 밖에 점 생성

# [6] 사분위수를 이용한 방법
# Q1 - 1.5*IQR 미만, Q3 + 1.5*IQR 초과 를 이상치로 판단 (IQR = Q3 - Q1)
# tip에 대한 이상치 구하기, 소수점 아래 2째자리까지 표기
s = tips['tip']
Q1, Q3 = s.quantile([0.25, 0.75])
IQR = Q3 - Q1
q_lower = Q1 - 1.5 * IQR
q_upper = Q3 + 1.5 * IQR
print(f'Lower: {round(q_lower,2)}, Upper: {round(q_upper,2)}')
# 소수점 아래 2째자리 까지 표기는 출력시에만 그렇지 않으면 이상치 대체할 때 박스플롯 밖에 점 생성

#### (2) 이상치 제거

In [ ]:
# 정상범주에 있는 데이터를 indexing 하는 방법
# [7] 이상치 제거 - 정상범주에 있는 데이터를 indexing 하는 방법으로 처리 (q_lower, q_upper 사이값이 정상)
condition1 = tips['tip'] >= q_lower
condition2 = tips['tip'] <= q_upper
tips2 = tips.loc[condition1 & condition2, 'tip']
print(tips.shape, tips2.shape)

# [8] 이상치 데이터 모음
condition1 = tips['tip'] <= q_lower
condition2 = tips['tip'] >= q_upper
tips_outlier = tips.loc[condition1 | condition2]
tips_outlier

#### (3) 이상치 대체

In [ ]:
# [9] 이상값 대체
# q_upper 보다 큰 데이터는 q_upper, q_lower 보다 작은 데이터는 q_lower로 변경
# tips3 데이터프레임에서 'tip' 값이 q_upper 보다 큰 것의 'tip'값을 q_upper 값으로 변경
# tips3 데이터프레임에서 'tip' 값이 q_lower 보다 작은 것의 'tip'값을 q_lower 값으로 변경
tips3 = tips.copy()
tips3.loc[tips3['tip'] > q_upper, 'tip'] = q_upper
tips3.loc[tips3['tip'] < q_lower, 'tip'] = q_lower

tips3['tip'].plot.box()
plt.show()
# 소수점 아래 2째자리 까지 표기는 출력시에만 그렇지 않으면 이상치 대체할 때 박스플롯 밖에 점 생성

#### (4) 데이터 변환
이상치를 완화하거나, 정규분포가 되도록 하기위해 사용, numpy의 log1p, sqrt, expm1, power 등의 함수 사용
- log1p, sqrt는 큰 값을 작게 만들어주며, 오른쪽 꼬리가 긴 근포를 정규분포로 변환하는데 사용, 큰 이상치를 작게 만들 수 있음
- expm1(exp마이너스원), power는 작은 값을 크게 만들어 주며, 왼쪽 꼬리가 긴 분포를 정규분포로 변환하는데 사용

In [ ]:
# [1] tips에서 tip의 분포 확인  (어느쪽 꼬리가 긴지 확인하기 위함)
s = tips['tip']
s.plot.hist(bins=20)    # bin은 몇개 구간으로 나눌지 나타냄
plt.show()

In [ ]:
# [2] y = log(x) 이해 -> np.power(np.e, y) = x  
# np.log(100) : log만 덩그러니 적혀있으면 자연로그e(np.e)로 해석,
# np.log10(100) : 10을 몇번 거듭제곱해야 100이라는 숫자가 되는지 구함 
# np.power(10, 2) : 앞의 수를 뒤의 수로 거듭제곱하면 나오는 수 구함
print(np.log10(100), np.power(10,2))
print(np.e, np.log(100), np.power(np.e, 4.605170185988092))

# [3] y = exp(x) 이해 -> y = power(np.e, x)
# exp(x)는 np.e에 x만큼 거듭제곱하라는 뜻
print(np.exp(4.605170185988092), np.power(np.e, 4.605170185988092))

# [4] log(0) -> x가 0인 경우 -inf 이기 때문에 x에 +1을 해서 동작하는 np.log1p를 사용함  // -inf는 무한대라는 뜻
np.log(0)

In [ ]:
# [5] tip의 분포를 오른쪽 꼬리가 긴 것에 대해서 짧게 만들기 (로그)
s = np.log1p(sri)   # 시리즈 데이터 상ㅂ입
s.plot.hist(bins=20)
plt.show()

# [6] tip의 분포를 오른쪽 꼬리가 긴 것에 대해서 짧게 만들기  (제곱근)
s = np.sqrt(tips['tip'])
s.plot.hist(bins=20)
plt.show()

# [7] tips['tip'] 원본 -> log1p -> expm1 = 원본 (로그 취했던 것)
a = tips['tip']
b = np.log1p(a)  # 변환
c = np.expm1(b)
print(a[:3], b[:3], c[:3], sep='\n\n')

# [8] tips['tip'] 원본 - sqrt - power = 원본 (제곱근 취했던 것)
a = tips['tip']
b = np.sqrt(a)
c = np.power(b, 2)
print(a[:3], b[:3], c[:3], sep='\n\n')

### 3) 파생 변수

#### (1) 포함된 목록 확인
- sri.isin() : 
- '찾을내용' in str : 

In [ ]:
df[df['category'].isin(['TV/방송', '게임'])]  # 또는 조건 isin으로도 해결 가능

# isin([]) 목록이 한개여도 리스트 타입으로 넣어줘야 함
df.loc[df['category'].isin(['음악/댄스/가수'])].sort_values('subscriber', ascending=False).head()

In [ ]:
# [4-26] bread에서 '사업장명' 컬럼을 사용하여
# '파리바게트', '파리바게뜨' 이름인 곳을 뽑아 paris로 이름 붙입니다.
bread[bread['사업장명'].str.contains('파리바게트', '파리바게뜨')]

#### (2) Accessor 사용
Series 타입에 .str을 붙여서 문자열 메소드 쓰는 것
- str.contains('문자열') : 특정 문자열을 포함하는지 아닌지를 True/False로 반환 (Boolean Indexing 조건으로 사용 가능)
- str.upper() : 영문자 소문자를 대문자로 변경  /  str.lower() : 영문자 대문자를 소문자로 변경  
- 세부 내용 : https://pandas.pydata.org/docs/reference/series.html#string-handling

Series의 데이터를 list 및 ndarray로 반환 (데이터프레임 말고 시리즈에서 변환하는 것)
- Series.to_list() : Series의 values를 list로 반환
- Series.to_numpy() : Series의 values를 ndarray로 반환 (Series.values 와 동일)

In [ ]:
# [1-44] title에 'KBS'가 포함된 채널 명 목록을 만들어 봅니다.
df[df['title'].str.contains('KBS')]
# 대소문자 구분 없이 검색 하려면? !str 두번 쓰면 됨
df[df['title'].str.lower().str.contains('kbs')]

# Series 형태를 list나 array 형태로 변환하기
df[df['title'].str.contains('KBS')]['title'].to_list()
df[df['title'].str.contains('KBS')]['title'].to_numpy()

In [ ]:
# datetime에서 년 정보만 가져오기
df['측정일시2'].dt.year
# datetime에서 월 정보만 가져오기
df['측정일시2'].dt.month
# datetime에서 일 정보만 가져오기
df['측정일시2'].dt.day
# 월요일이 0 일요일이 6
df['측정일시2'].dt.dayofweek

In [ ]:
# [4-8] bread 의 '소재지전체주소' 중 시/도에 대한 정보(목록)를 추출합니다.
bread['소재지전체주소'].str.split(' ').str[0]   # str 여러번 붙여 써도 괜찮음

# [4-9] bread에서 소재지전체주소의 처음이 '서울특별시'이면서,
# '업태구분명'이 '제과점영업'인 것만 추출합니다.
bread[(bread['업태구분명']=='제과점영업') & (bread['소재지전체주소'].str.split(' ').str[0]=='서울특별시')]
# 이렇게 정리해도 괜찮은 듯
condition1 = bread['업태구분명']=='제과점영업'
condition2 = bread['소재지전체주소'].str.split(' ').str[0]=='서울특별시'
bread = bread[condition1 & condition2]

- str.strip('제거할 문자들') : 문자열의 앞/뒤에 불필요한 것을 제거함
   - 제거할 문자들을 지정하지 않을 경우 whitespace를 제거함
- str.split('구분자')
   - 구분자를 지정하지 않을 경우 whitespace를 기준으로 분리함
   - 각 구분된 내용은 str[0], str[1], .. 등으로 접근
- str.join('구분자')
   - 구분자 지정을 생략할 수 없음
   - 분리된 문자열을 구분자를 사이에 넣어 하나의 문자열로 만듦
- str.replace(전, 후)
   - 문자열의 일부 내용을 변경 가능함
   - 변경전 내용을 찾아 변경후 내용으로 바꿈   


In [ ]:
data = {'A': ['    김   수민 ', '  이  나라     ', '  황   소  라  '],
        'B': ['  d2021-10-29.   ', '   \n\t\r2021-10-30c    \n', '2021-11-01c   '],
        'C': ['*7', '6', '*7']}
temp = pd.DataFrame(data)
print(temp)

# [1] 'A' 컬럼의 데이터를 빈칸 없는 이름으로 만들어 temp의 'A'컬럼 뒤에 'A-01'컬럼으로 추가해 보세요.
temp.insert(1, column=' A-01', 
            value=temp['A'].str.split().str.join(''))        

# [2] 'B' 컬럼의 데이티를 2021-10-29 처럼 앞/뒤에 공백이나 다른 문자('.dc')가 없도록 만들어
# temp에 'B-01' 컬럼으로 추가해 보세요.
temp.insert(3, column='B-01', 
            value=temp['B'].str.strip().str.strip('d.c'))

# [3] 'B-01' 컬럼의 데이터에서 '-'를 '/'로 수정해 temp에 'B-02' 컬럼으로 추가해 보세요.
temp.insert(4, column='B-02',
            value=temp['B-01'].str.replace('-', '/'))

# [4] 'C' 컬럼에서 *을 제거하고 숫자로 변경해 'C-01'컬럼으로 추가해 보세요.
temp['C-01'] = temp['C'].str.replace('*','').astype(int)


#### (3) 불리언 인덱싱
- df.loc[조건] : 조건은 boolean dtype이어야함 (기호 : | 선언, & 연언, ~ 부정) !!참 거짓으로 나올 수 있어야함
- df.iloc[행범위, 열범위] : 인덱스 번호로 인덱싱 혹은 슬라이싱

In [ ]:
# [예시 1-36] 'category'가 '음악/댄스/가수'인 채널의 subscriber TOP5를 알아봅니다.
df.loc[df['category']=='음악/댄스/가수'].sort_values('subscriber').head()

# [2-23] 와인 소비량이 맥주 + 증류주 소비량보다 큰 나라를 검색해,'대륙'을 기준으로 정렬해 보자
df.loc[(df['와인']) > (df['맥주']+df['증류주'])].sort_values('대륙')

# [2-24] 맥주 소비량이 230 초과이면서, 와인 소비량이 230 초과인 나라를 검색해 보자
df.loc[(df['맥주']>230) & (df['와인']>230)]['국가']

# [2-25] 대륙이 'AS'인 국가들의 정보를 검색해 보자
df.loc[df['대륙']=='AS'].info()

- 참 거짓으로 나오지 않는 사칙연산 같은 건 loc 쓰지 않음

In [ ]:
# [2-26] 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량')를 추가합니다
# 주류소비량 = '맥주' + '증류주' + '와인'
df['주류소비량'] = (df['맥주'])+(df['증류주'])+(df['와인'])

# [2-27] 주류소비량2 = ['맥주', '증류주', '와인']에 대해 DataFrame.sum(axis=1) 함수 사용
df['주류소비량2'] = df[['맥주', '증류주', '와인']].sum(axis=1)

#### (4) 추가, 제거 및 병합
- 행 추가 : df.append(추가할 데이터프레임)
- 컬럼 추가1 : df['새로운칼럼명'] = 
- 특정 위치에 컬럼 추가2 : df.insert(위치, 컬럼, 값) 단, inplace 동작됨
- 열 제거1 : df = df.drop('열이름')    
- 열 제거2 : df = df.drop(rows=['열이름1', '열이름2' ...])
- 컬럼 제거1 : df = df.drop('칼럼명', axis=1)
- 컬럼 제거2 : df = df.drop(columns=[컬럼명...])
- 컬럼 제거3 : del df['제거할칼럼명'] 단, inplce 동작됨
- 데이터프레임 행/열 전환 : df.T

In [ ]:
# [2-26] 국가별 주류 소비량 합계를 구해 새로운 컬럼 ('주류소비량')를 추가합니다
# 주류소비량 = '맥주' + '증류주' + '와인'
df['주류소비량'] = (df['맥주'])+(df['증류주'])+(df['와인'])

# [2-40] 세계의 각 컬럼별 평균을 구하여 DataFrame으로 만들고,
# worldwide라는 이름을 지정합니다
# 세계의 각 컬럼별 평균은 DataFrame.mean()을 사용합니다.
worldwide = pd.DataFrame(df.mean(axis=0))

# [2-41] worldwide의 행과 열을 전환해 wwT로 저장합니다.
wwT = worldwide.T
wwT

# [2-42] wwT의 맨 앞에 '국가' 컬럼을 'World Wide' 값으로 추가합니다.
# 여러 번 추가하면 안됨
wwT.insert(0,'국가','World Wide') # 값을 여러개 줄 땐 리스트로 묶어줘야 한다.
wwT

- 행 병합 : concat([합칠행1, 합칠행2] ..., axis=0)
- 열 병합 : concat([합칠열1, 합칠열2] ..., axis=1)
* df.append()와 달리 여러 개의 df를 목록을 주어 한 번에 합칠 수 있음

In [ ]:
# [2-44] wwT와 korea 를 합쳐 하나의 DataFrame을 생성하여 df2로 저장합니다.
df2 = pd.concat([wwT,korea], axis=0)
df2

# [3-5] df2016, df2017, df2018, df2019를 합쳐 한 개의 DataFrame으로 만들어 df라는 이름을 지정합니다.
dfList = [df2016, df2017, df2018, df2019]
df = pd.concat(dfList, axis=0)

# concat을 하더라도 인덱스 번호는 유지되기 때문에 인덱스번호 다시 설정해줘야함
df.index = pd.RangeIndex(len(df))
# concat 할때부터 옵션으로 인덱스 잡아주기
df = pd.concat(dfList, ignore_index=True, axis=0)

In [ ]:
# [4-20] bread에 '설립년도' 및 '폐업년도' 컬럼을 추가합니다.
# '인허가일자'//10000, '폐업일자 // 10000 을 사용하여 구합니다.
# 두 개의 컬럼이 추가된 bread의 첫 2개 행을 확인합니다.

# 정수형인경우
year = bread['인허가일자'] // 10000
meonth = bread['인허가일자'] // 100 % 100
day = bread['인허가일자'] %100

# datetime 으로 바꿔서
temp = pd.to_datetime(bread['인허가일자'], foramt='%Y,%m%d')
temp.dt.year
temp.dt.moth
temp.dt.day

# 문자열인경우 (슬라이싱으로)


# 이번에는 정수형으로 함
bread['설립년도'] = bread['인허가일자']//10000
bread['폐업년도'] = bread['폐업일자']//10000
bread.head()

In [ ]:
# [4-21] bread에 '영업기간' 컬럼을 추가합니다
# '영업기간'은 '상태코드'가 1(=영업)인 경우 2021 - 설립년도 +1
# '상태코드'가 2(=폐업)인 경우 폐업년도 - 설립년도 + 1로 계산합니다.

from datetime import datetime
today = datetime.today()
today.year
today.month
today.day

nyear = today.year
bread.loc[bread['상태코드']==1,'영업기간'] = nyear - bread['설립년도'] + 1
bread.loc[bread['상태코드']==2, '영업기간'] = bread['폐업년도'] - bread['설립년도'] + 1
bread.head(3)

In [ ]:
# [4-22] bread의 '설립년도'별 데이터 수를 구해 년도별로 정렬하고,
# DataFrame으로 변경하여 전치행렬을 구해 temp1 이름을 부여해 출력합니다.
temp1 = bread['설립년도'].value_counts().sort_index(ascending=True)
temp1 = pd.DataFrame(temp1).T

In [ ]:
# [4-28] 설립년도가 2000년 이후이면서 영업 중인 곳의 영업기간 정보를 구합니다.
# paris, tous에 대해 각각 구해서 temp1, temp2로 이름 붙입니다.
temp1 = paris.loc[(paris['설립년도']>=2000) & (paris['상태코드']==1), '영업기간']
temp2 = tous.loc[(tous['설립년도']>=2000) & (tous['상태코드']==1), '영업기간']
# temp1, temp2의 평균을 구해 이름을 comp로 하는 DataFrame으로 만듭니다.
# index => ['파리바게트', '뚜레쥬르'], columns => ['영업']
# 선생님방법
s = pd.Series([temp1.mean(), temp2.mean()], index=['파리바게트', '뚜레쥬르'])
comp = pd.DataFrame(s, columns=['영업'])
comp

# [4-29] 설립년도 2000년 이후이면서 폐업한 곳의 영업기간 정보를 구합니다.
# paris, tous에 대해 각각 구해서 temp1, temp2로 이름 붙입니다.
temp1 = paris.loc[(paris['설립년도']>=2000) & (paris['상태코드']==2), '영업기간']
temp2 = tous.loc[(tous['설립년도']>=2000) & (tous['상태코드']==2), '영업기간']
# temp1, temp2의 평균을 구해 comp에 '폐업' 컬럼으로 추가합니다.
comp['폐업']= [temp1.mean(), temp2.mean()]
comp

# [4-31] other의 2000년 이후 설립된 곳의 영업, 폐업 사업장을 구한 뒤
# temp1, temp2 이름을 붙입니다.
temp1 = other.loc[(other['설립년도']>=2000) & (other['상태코드']==1), '영업기간']
temp2 = other.loc[(other['설립년도']>=2000) & (other['상태코드']==2), '영업기간']
# temp1, temp2의 평균을 구해 comp 에 '나머지' 행으로 추가합니다.
temp = pd.DataFrame([[temp1.mean(), temp2.mean()]], index=['나머지'], columns=['영업', '폐업'])
comp = comp.append(temp)
comp


## 7. 데이터 표본 추출(샘플링)

- df.sample(n=개수, frac=비율, random_state=None, ignore_index=False)

In [ ]:
# 단순 무작위 추출
df.sample(frac=0.02, random_state=777)
df.sample(n=5, random_state=777)

# 계통 추출 : 시작 위치부터 일정한 간격으로 추출하는 것 
df.iloc[::50,:]   # 인덱스 50단위로 계통 추출

# 층화 추출 : 독립변수의 unique값이 실제 분포하는 비율에 맞춰서 데이터 추출
# [5] 층화 추출 - 성별을 기준으로 성별 비율에 맞춰 10개 데이터 추출
# [5-1] 성별 비율 구하기(rate), 추출할 sample 개수 구하기(sample_n)
sample_n = 20   # 전체적으로 사용할 샘플의 개수
temp = tips['sex'].value_counts(normalize=True).to_frame()
temp.columns=['rate']
temp['sample_n'] = round(sample_n * temp['rate'], 0).astype('int')
# [5-2] Female, Male로 데이터 분리 및 sample개수 만큼의 임의 표본 추출
Female = tips.loc[tips['sex']=='Female']
Male = tips.loc[tips['sex']=='Male']
df1 = Female.sample(n=temp.loc['Female', 'sample_n'], random_state=1)
df2 = Male.sample(n=temp.loc['Male', 'sample_n'], random_state=1)
df = pd.concat([df1, df2])
df.sample(frac=1, random_state=1)   # 섞기 위해서 frac=1 사용

## 8. 도수분포표

관측 값을 몇 개 범주로 나눈 다음 그 범주에 속하는 관측 값의 개수(도수)와 그 도수를 전체 관측 값의 개수로 나눈 값(상대도수)에 대한 표

In [ ]:
# [1] 데이터 생성하기
data = ['A+', 'A', 'B+', 'B', 'C+', 'C', 'D+', 'D', 'F']
cnt  = [3, 6, 12, 16, 10, 8, 4, 1, 2]
mylist = []
for s, c in zip(data, cnt):
    mylist.extend([s] * c)

df = pd.DataFrame(mylist, columns=['data'])
df = df.sample(frac=1)
print(df['data'].to_list())

# [2] mylist를 사용하여 돗수분포표를 만들어 본다
s = pd.Categorical(df['data'], categories=data, ordered=True)
s = s.value_counts().sort_index()
print(s)

# [3] s를 사용하여 돗수분포표의 비율, 누적인원, 누적비율을 완성해 본다
df = pd.DataFrame(s, columns=['인원'])
df.index.name = '학점'
df['비율'] = (df['인원']/ df['인원'].sum()).round(2)
df['누적인원'] = df['인원'].cumsum()
df['누적비율'] = df['비율'].cumsum()
print(df)

## 9. 상관계수

In [ ]:
# [0] 데이터 불러오기
# sns.load_dataset(데이터셋의 이름)
# iris(붓꽃) 데이터를 불러옴
# sepal : 꽃받침, petal : 꽃잎
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

# [1] species를 LabelEncoding하여 'species_LE' 컬럼 추가하기
from sklearn.preprocessing import LabelEncoder
ans = LabelEncoder().fit_transform(iris['species'])

# [2] iris의 상관계수 구하기
iris['species'] = LabelEncoder().fit_transform(iris['species'])

iris.corr()

# 유형2 : 데이터 모델링 1문제 * 40 [40점]
- pandas 및 sklearn을 활용한 머신러닝 작업에 대한 능력 확인
- 주어진 데이터를 활용해 전처리, 모형 구축, 평가 작업을 한 뒤
- 예측 결과를 csv파일로 저장

## 1. 데이터 스케일링(수치형 데이터 스케일링)

min-max normalization : 값의 범위를 [0, 1]로 변환
- (xi - x.min()) / (x.max() - x.min())  
  
standardiztion : 특성의 값이 표준 정규분포를 갖도록 변환 (평균 0, 표준편차 1)
- (xi - x.mean()) / x.std()

In [ ]:
# [1] Scaling을 위한 함수 구현
def minmaxScale(x):
    return (x - x.min()) / (x.max() - x.min())

def standardScale(x):
    return (x - x.mean()) / x.std()

# [2] minmaxScale 함수를 사용한 'tip' 컬럼의 스케일링
x = minmaxScale(tips['tip'])

# [3] standardScale 함수를 사용한 'tip' 컬럼의 스케일링
x = standardScale(tips['tip'])

In [ ]:
# [4] sklearn 라이브러리의 스케일러(MinMaxScaler)를 사용한 스케일링
# 스케일러의 fit_transform() 사용시 2차원의 데이터를 전달해야 함
# (DataFrame도 2차원), 결과는 ndarray로 반환 됨
from sklearn.preprocessing import MinMaxScaler, StandardScaler
x = MinMaxScaler().fit_transform(tips[['tip']])     # 이차원으로된 데이터를 넣어줘야하고, fit_transform하는 것을 까먹지 않으면 제일 편한 방법
print(x.min(), x.max())

# [5] sklearn 라이브러리의 스케일러(StandardScaler)를 사용한 스케일링
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(tips[['tip']])
print(x.mean(), x.std())

In [ ]:
# [6] scipy.stats의 zscore 함수를 사용한 스케일링
from scipy.stats import zscore
x = zscore(tips['tip'])
print(x.mean(), x.std())

## 2. 데이터 인코딩(범주형 데이터 라벨링) 

학습을 위해 범주형 데이터를 수치형으로 바꿔준다.  
  
label Encoding : 값의 일련번호로 표기함 : 순서가 있는 경우 유리, 값이 2개 밖에 없는 경우
- sri.replace() 사용  

one hot encoding : 범주의 개수 만큼 feature를 만들어냄 : 순서가 없는 경우 유리
- pd.get_dummies(sri)
- pd.get_dummies(df)

In [ ]:
import pandas as pd
df = pd.DataFrame({'A':['월', '화', '수', '화', '수', '금', '월'],
                   'B':['여자', '남자', '여자', '남자', '아이', '남자', '아이']})

# [1] 요일에 대해서 Label Encoding 합니다.  (cat.code 사용)
weekdays = '월 화 수 목 금 토 일'.split()
print(weekdays)
df['A_LE'] = pd.Categorical(df['A'], weekdays, ordered=True)
df['A_LE'] = df['A_LE'].cat.codes

# [2] '남자', '여자', '아이'에 대해서 Label Encoding 합니다 (replace 사용)
v = df['B'].unique()
df['B'].replace(v, range(len(v)))

# [3] df의 'A', 'B' 컬럼을 One Hot Encoding 합니다.
a = pd.get_dummies(df['A'])
b = pd.get_dummies(df['B'])
df2 = pd.concat([a,b], axis=1)

# [6] df의 모든 범주형 변수를 OneHotEncoding 합니다
df3 = pd.get_dummies(df)

## 3. Binning (연속형 변수를 구간을 이용해 범주화)


연속형 변수를 구간을 이용해 범주화하여 정보를 압축시키고 단순하게 만듬 (정확도는 떨어짐)  
이상치 해결방법 중 한가지로 사용하거나, 오버피팅 방지 기법으로 사용되기도 함
- (3,6] : 3초과 6이하, right=True
- [3, 6) : 3이상, 6미만, right=False
- pd.cut

In [ ]:
# [0] 데이터 준비
data = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
df = pd.DataFrame(data, columns=['data'])
df['data'].to_list()

# [1] data를 사용하여 0~3, 4~6, 7~10 에 대한 binning을 하여 보도록 한다
# 범주의 레이블은 ['A', 'B', 'C']를 사용한다

# (Min-1, 3], (3, 6], (6, Max] 로 범주를 만들어 result_A 컬럼으로 추가합니다.
df['result_A'] = pd.cut(df['data'], [df['data'].min()-1,3,6,10], labels=['A', 'B', 'C'], right=True)
# [Min, 3), [3, 6), [6, Max+1) 로 범주를 만들어 result_B 컬럼으로 추가합니다.
df['result_B'] = pd.cut(df['data'], [df['data'].min(),3,6,df['data'].max()+1], labels=[0, 1, 2], right=False)

In [ ]:
# [2] result_A에 대해 LabelEncoding을 하여 result_C를 컬럼으로 추가합니다.
from sklearn.preprocessing import LabelEncoder
df['result_C'] = LabelEncoder().fit_transform(df['result_A'])

비율을 사용하여 구간 나누기
- pd.qcut(데이터, 구간)
- 구간은 0~1 사이의 숫자 사용

In [ ]:
# [3] 비율을 사용하여 구간나누기
df = pd.DataFrame([0, 1, 2, 3, 10, 11, 12, 13, 20, 30, 40, 50], columns=['data'])
pd.qcut(df['data'], [0, 0.25, 0.5, 0.75 ,1])  # 네 구간으로 나눈 것

# [4] 다음 데이터를 binning 하여보세요 (bins 컬럼 추가)
# 사용구간 : [10, 40), [40, 50), [50, 60), [60, 70), [70, 80), [80, 101)
# 사용레이블 : '10-40미만', '40-50미만', '50-60미만', '60-70미만', '70-80미만', '80-100이하'
data = [55.6, 83.3, 43.4, 58.1, 31.6, 55.6, 60.7, 64.6,
        73.3, 55.6, 64.3, 52.8, 22.7, 46.3, 71.4, 53.8,
        64.5, 67.9, 71.4, 80.0, 59.5, 40.5, 77.1, 58.6,
        65.4, 52.4, 66.7, 91.3, 41.3, 72.1, 61.9, 78.4,
        63.6, 41.0, 65.2, 81.3, 54.8, 19.6, 50.0, 53.1,
        41.2, 56.5]
df = pd.DataFrame(data, columns=['data'])
df['bins'] = pd.cut(df['data'], [10, 40, 50, 60, 70, 80, 101],
       labels = ['10-40미만', '40-50미만', '50-60미만', '60-70미만', '70-80미만', '80-100이하'], 
       right=False)
# [5] bins 컬럼에 대해 LabelEncoding을 하여 bin_n 컬럼을 추가합니다.
df['bins'] = df['bins'].cat.codes

# [6] 다음 나이 데이터를 binning 하여보세요
bin_labels = 'Baby Child Teenager Student Young_Adult Adult Elderly'.split()   # 7개 label
data = [42, 11, 40, 16, 35, 58, 1, 13, 22, 7, 62, 11, 52, 67, 42, 33, 15, 60, 36, 36]
df = pd.DataFrame(data, columns=['age'])

# (-1, 5], (5, 12], (12, 18], (18, 25], (25, 35], (35, 60], (60, df['age'].max()] 로
# 범주를 만들어 age_cat 컬럼으로 추가합니다.
df['age_cat'] = pd.cut(df['age'], [-1, 5, 12, 18, 25, 35, 60, df['age'].max()], 
                       labels = bin_labels,
                       right = True)
df['age_LE'] = df['age_cat'].cat.codes

## 4. 모델링

### 1) 분류

#### (1) 이진분류

#### (2)다중분류

### 2) 분류 평가방법


- 정확도(accuracy) : 
- 정밀도(precision) : 
- 재현율(recall) : 
- 특이도(Specificity) : 
- f1_score : 
- ROC(Receiver Operating Characteristic) 곡선 :  
- AUC(Are Under Curve) : 

### 3) 회귀

### 4) 회귀 평가방법

- SSE(Sum Squared Error) : 
- MSE(Mean Squared Error) : 
- RMSE(Root Mean Squared Error) :
- MAE(Mean Absolute Error) : 
- 결정계수(R2) :
- Adjusted R2 : 
- MSPE(Mean Squared Percentage Error) : 
- MAPE(Mean Absolute Percentage Error) :
- RMSLE(Root Mean Squared Logarithmic Error) : 
- AIC(Akaike Information Criterion) : 
- BIC(Bayes Information Criteria) : 


# 유형3 : 통계 검증 2문제 * 15 [30점]
- scipy 모듈을 활용한 통계적 유의성 검정에 대한 능력 확인
- 주어진 데이터를 활용해 가설검정 진행 후, 통계량, p-value, 채택/기각 등의 결과 출력
- 소수점 아래 자릿수가 정해진 실수로 답을 출력

## 1. 확률 분포

확률 현상 : 나오는 결과의 범위는 알지만, 가능한 결과들 중 정확히 어떤 결과가 나올지 모르는 현상
확률 변수 : 확률 현상에 기인해 결과 값이 확률적으로 정해지는 변수    
확률 분포 : 확률 변수가 특정한 값을 가질 확률을 나타내는 분포

확률 질량 함수 (PMF) : 이산 확률 변수에서 특정 값에 대한 확를을 계산하기 위한 함수 (이산형에서 사용)  
확률 밀도 함수 (PDF) : 연속 확률 변수에서 특정 구간에 속할 확률을 계산하기 위한 함수 (연속형에서 사용)   
누적 분포 함수 (CDF) : 어떤 확률 분포에 대해 확률 변수가 특정 값보다 작거나 같을 확률을 계산하기 위한 함수 (이산형과 연속형에서 사용)  
(PPF) : CDF 반대, 확률 값을 주면 그 때의 x값을 줌

### 1) 이산형 확률분포 : 확률변수가 몇 개의 한정된 가능한 값을 가지는 분포 (각 사건은 서로 독립이어야 함)  
#### (1)베르누이 분포 : 확률 변수의 가능한 값이 두 개뿐인 이산 확률 분포, 이러한 실험 1회 시행이 베르누이 시행이라 함
#### (2) 이항 분포 : 연속된 n번의 독립적 시행에서 각 시행이 확률 p를 가질 때의 이산 확률 분포 (n이 1일 때 이항 분포는 베르누이 분포와 같다)
#### (3) 기하 분포 : 성공/실패 로 구성된 시행을 연달아 수행시 처음 성공할 때 까지 시도한 회수 k에 대한 
#### (4) 초기하 분포 : 
#### (5) 포아송 분포 : 

### 2) 연속형 확률분포 : 확률변수의 가능한 값이 무한 개 이며 사실상 셀 수 없는 경우의 분포
#### (1) 정규 분포(가우스 분포) : 
#### (2) 표준 정규 분포 (Z-분포)
#### (3) T-분포 : 
#### (4) 카이제곱 분포(x제곱-분포) :
#### (5) F-분포 
#### (6) 지수 분포 : 
#### (7) 감마 분포 : 


### 3) 신뢰구간
#### (1) 신뢰수준 : 추정값이 존재하는 구간에 모수가 포함되어 있을 가능성의 크기 또는 정확도
#### (2) 신뢰구간 : 신뢰수준을 기준으로 추정된 통계적으로 유의미한 모수의 범위  
- 표본평균 - (z * SE) ~ 표본평균 + (z * SE)  
    - SE ; 표준오차

## (중요!) 2. 가설 검정
귀무 가설(H0) : 현재까지 주장되어 온 것이나 기존과 비교하여 차이가 없음을 나타내는 가설  
대립 가설(H1) : 표본을 통해 확실한 근거를 가지고 입증하고자 하는 가설(연구 가설)  
  
p-value : 귀무 가설이 참이라는 가정하에 관찰된 결과가 일어날 확률 값   
- p-값 0.05 이상 : 귀무가설 채택, 대립가설 기각
- p-값 0.05 미만 : 귀무가설 기각, 대립가설 채택
  
가설 검정 오류  
- 제 1종 오류 : 실제(귀무가설 사실) , 판단(귀무가설 기각)
- 제 2종 오류 : 실제(귀무가설 거짓) , 판단(귀무가설 채택)



## (중요!) 3. 모수 검정 -> 그룹 나누는 법을 알아야 한다
표집분포의 모수를 알고 있다고 가정

### 1) 정규성 검정 : 정규분포인지 검증
- 귀무가설 : 데이터셋이 정규분포를 따른다
- 대립가설 : 데이터셋이 정규분포르 따르지 않는다.
- 유의 수준 0.05인 경우, p-value >= 0.05 정규성이 보장된다고 할 수 있다.
- 검정통계량 < 임계값 인 경우, 정규성이 보장된다고 할 수 있다. (검정 통계량이 임계값 이하? 미만? 인 경우)

In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv('./eduatoz/04. 통계적 검정/data_02/sleepage.csv')
df.info()

# 그룹을 나누어 저장함
gA = df['stime20s']
gB = df['stime40s']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   stime20s  20 non-null     int64
 1   stime40s  20 non-null     int64
 2   ID        20 non-null     int64
dtypes: int64(3)
memory usage: 608.0 bytes


In [28]:
# 정규성 검정 - 1 shapiro wilks test # 표본 5000개 미만 
from scipy.stats import shapiro  
statisticA, pvalueA = shapiro(gA)
statisticB, pvalueB = shapiro(gB)

# f-string으로 자리수 맞춰서 출력하기
print(f'statisticA:{statisticA:.4f}, pvalueA{pvalueA:.4f}') # 정규성 만족
print(f'statisticB:{statisticB:.4f}, pvalueB{pvalueB:.4f}') # 정규성 만족

statisticA:0.9777, pvalueA0.4595
statisticB:0.9778, pvalueB0.4647


In [13]:
# 정규성 검정 - 2 kstest(kolmogorov-smirnov test) # 표본수 2000 초과 // 정규성 검정에만 쓰는 건 아님
from scipy.stats import kstest # 두 개의 데이터가 같은 분포인지 아닌지 검정 
# 귀무가설 : 두집단의 분포가 같다
# 대립가설 : 두집단의 분포가 다르다

print(kstest(gA, "norm")) # 비교대상으로 "norm"으로 정규분포 데이터를 준다 / 정규성 불만족
print(kstest(gB, "norm")) # 정규성 불만족

KstestResult(statistic=0.9986501019683699, pvalue=8.073266280952718e-58)
KstestResult(statistic=0.9999997133484281, pvalue=2.8061866176047734e-131)


In [14]:
# 정규성 검정 - 3 normaltest  # 표본 20개 이상
from scipy.stats import normaltest

print(normaltest(gA))  # 정규성 만족
print(normaltest(gB))  # 정규성 만족

NormaltestResult(statistic=1.173158148726697, pvalue=0.5562268444276741)
NormaltestResult(statistic=2.045029526680344, pvalue=0.3596892684891455)


In [24]:
# 정규성 검정 - 4 anderson darling test # 검정통계량, 임계값 출력 (검정통계량 < 임계값이면 귀무가설 채택) // 정규성 검정에만 쓰는 건 아님
# 검정통계량 < 임계값, 정규성이 보장된다고 할 수 있다
from scipy.stats import anderson
rA = anderson(gA, dist="norm") # static(검정통계량), critical_values(임계값), significance_level(유의수준)
rB = anderson(gB, dist="norm")

print('그룹A:', *rA)
print('그룹B:', *rB)

그룹A: 0.4079859754958619 [0.538 0.613 0.736 0.858 1.021] [15.  10.   5.   2.5  1. ]
그룹B: 0.360841172628966 [0.538 0.613 0.736 0.858 1.021] [15.  10.   5.   2.5  1. ]


### 2) 등분산성 검정 : 분산이 동일한지 검증
* 귀무가설 : 데이터셋이 등분산성을 충족한다.
* 대립가설 : 데이터셋이 등분산성을 충족하지 않는다.

- burtlett : 정규성 충족하는, 데이터셋의 크기가 서로 다른 2개 이상의 집단 사용 가능
- levene, fligner : 정규성을 충족하지 않는, 비모수 데이터에 대해서도 사용가능 (중앙을 median으로 설정)

In [ ]:
from scipy.stats import bartlett, levene, fligner
bartlett()
levene() # center={'median'(정규분포가아닌경우사용), 'mean'(좌우대칭일때사용), 'trimmed'(양 끝단에 데이터가 헤비하게 있을 때 사용)}, proportiontocut=0.05 center='trimmed'일 경우 사용
fligner()

In [23]:
# 파일 읽어와 내용 확인
import seaborn as sns
iris = sns.load_dataset('iris')
print(iris.head(2))

# [1] target='sepal_length', 품종별 그룹을 나누어 저장함
import pandas as pd
target = 'sepal_length'
iris['species'].unique()
gA = iris.loc[iris['species']=='setosa', target]
gB = iris.loc[iris['species']=='versicolor', target]
gC = iris.loc[iris['species']=='virginica', target]

# [2] burtlett 검정
from scipy.stats import bartlett, shapiro
# 정규성검정
_, pvalueA = shapiro(gA)
_, pvalueB = shapiro(gB)
_, pvalueC = shapiro(gC)
print(pvalueA, pvalueB, pvalueC)    # p-value가 0.05 이상 귀무가설 채택 - 정규성 만족
# 등분산성 검정
static, pvalue = bartlett(gA, gB, gC)
print(static, pvalue)   # p-value가 0.05 미만 - 귀무가설 기각 - 등분산성 불만족

# [3] levene 의 center는 'mean'으로 지정
from scipy.stats import levene
print(levene(gA, gB, gC, center='median')) # p-value가 0.05 미만 - 귀무가설 기각 - 등분산성 불만족

# [4] fligner의 center는 'trimmed', proportiontocut=5% 지정
from scipy.stats import fligner
print(fligner(gA, gB, gC, center='trimmed'))    # p-value가 0.05 미만 - 귀무가설 기각 - 등분산성 불만족

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa


### 3) T-검정
- (1) One sample T test : 1표본 평균 검정 : ttest_1samp(a, popmean, alternative='') 이다아니다 = 'two-sided', 작다='less', 크다='greater'
- (2) Paried T test : 쌍체(대응) 표본 t-검정 : ttest_rel(a, b, alternative='two-sided)
- (3) Two sample T test : 2표본 평균 검정 : ttest_ind(a, b, alternative='two-sided', equal_var=True/False) 등분산아닌 경우 equal_var=False


### 4) ANOVA
- 일원분산분석(One-Way ANOVA)범주형 독립변수가 한 개인 경우 사용 : f_oneway(*data)
- 이원분산분석(Two-Way ANOVA)주형 독립변수가 두 개인 경우 사용(K-Way ANOVA; 범주형 변수가 K개인 경우 사용) <- 나올확률 낮은 듯


## (중요!) 4. 비모수 검정 -> 교차표 만드는 법 알아야함
표집분포의 모수를 알지 못한다고 가정
### 1) 카이검정
- 적합도 검정 : 
- 동질성 검정 : 
- 독립성 검정 : 

데이터 검증하기  
1) t-검정  
2) 정규성 검정  
3) One-way ANOVA  
4) Two-way ANOVA  
5) 카이제곱 검정  
(1) 독립성 검정  
(2) 동질성 검정  
(3) 적합성 검정  
6) 순위 검정  
(1) 윌콕슨 순위합 검정  
(2) 크루스칼 - 왈리스 검정  
07) 부호검정  
(1) 부호검정  
(2) 맥네마 검정  
8) 다변량 추론  
(1) 다중 회귀 분석  